In [114]:
from scipy import misc, sparse
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from skimage import io
import warnings
import json
warnings.filterwarnings('ignore')
from shapely.geometry.polygon import LinearRing, Polygon
from shapely.geometry.point import Point
from shapely.ops import cascaded_union
import pickle
from itertools import compress
from ast import literal_eval

from IPython.display import display, HTML

with open('atlas/atlas_with_mask.pickle', 'rb') as infile:
    atlas = pickle.load(infile)
    
area_threshold = 0
atlas_with_poly = [poly for poly in atlas if 'poly_mask' in poly and len(poly['poly_mask'][0]) > area_threshold]

polys = [np.array(shape['poly_mask']) for shape in atlas_with_poly]
polys_packed = [poly[0]*1000+poly[1] for poly in polys]

def get_subreddits(subreddits):
    return [x.strip().strip(',').strip('/').lstrip('r/').lower() for x in subreddits.split()]

I simply used the list of valid pixel inside a shape to find the area

In [116]:
subreddit_shapes = {}
for idx, shape in enumerate(atlas_with_poly):
    for subreddit in get_subreddits(shape['subreddit']):
        if not subreddit in subreddit_shapes:
            subreddit_shapes[subreddit] = set()
        subreddit_shapes[subreddit].update(polys_packed[idx])
subreddit_areas = {k: len(v) for k,v in subreddit_shapes.items()}

df_area = (pd.DataFrame.from_dict(subreddit_areas, orient='index')
             .reset_index()
     .rename(columns={0: 'area', 'index': 'subreddit'})
     .sort_values(by='area', ascending=False))

with pd.option_context('display.max_rows', None):
    display(df_area)

,subreddit,area
246,ainbowroad,87342
74,placenl,33059
69,prequelmemes,21406
44,sweden,19120
58,placehearts,18672
22,placede,17078
21,de,17002
45,thenetherlands,16870
67,2007scape,15487
3,monalisaclan,15071


Sanity checking with polygons, overall it works, outliers are because of badly formed polygon

In [124]:
subreddit_shapes_poly = {}
for idx, shape in enumerate(atlas_with_poly):
    for subreddit in get_subreddits(shape['subreddit']):
        if not subreddit in subreddit_shapes_poly:
            subreddit_shapes_poly[subreddit] = []
        subreddit_shapes_poly[subreddit].append(shape['poly'])
subreddit_areas_poly = {subreddit:
                        cascaded_union(
                            [shape if shape.is_valid else shape.buffer(0) for shape in shapes]
                        ).area
                        for subreddit,shapes in subreddit_shapes_poly.items()}

df_area_poly = (pd.DataFrame.from_dict(subreddit_areas_poly, orient='index')
             .reset_index()
     .rename(columns={0: 'area', 'index': 'subreddit'})
     .sort_values(by='area', ascending=False))

(df_area_poly.set_index('subreddit') - df_area.set_index('subreddit')).abs().describe()

,area
count,788.000000
mean,9.519684
std,55.732038
min,0.000000
25%,0.000000
50%,0.785947
75%,4.500000
max,1086.000000
